In [26]:
import numpy as np
import pickle

# import data.load
from metrics.accuracy import conlleval

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, GRU, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, MaxPooling1D

import progressbar

## Load the data

In [27]:
### Load Data
with open('atis.pkl', 'rb') as f:
    train_set, valid_set, test_set, dicts = pickle.load(f)

w2idx, ne2idx, labels2idx = dicts['words2idx'], dicts['tables2idx'], dicts['labels2idx']

# Create index to word/label dicts
idx2w  = {w2idx[k]:k for k in w2idx}
idx2ne = {ne2idx[k]:k for k in ne2idx}
idx2la = {labels2idx[k]:k for k in labels2idx}

In [28]:
print(len(train_set), len(train_set[0]))

3 19915


In [29]:
### Ground truths etc for conlleval
train_x, train_ne, train_label = train_set
val_x, val_ne, val_label = valid_set

words_val = [ list(map(lambda x: idx2w[x], w)) for w in val_x]
groundtruth_val = [ list(map(lambda x: idx2la[x], y)) for y in val_label]
words_train = [ list(map(lambda x: idx2w[x], w)) for w in train_x]
groundtruth_train = [ list(map(lambda x: idx2la[x], y)) for y in train_label]

In [30]:
print(len(train_label))

19915


## Build the model

In [31]:
### Model
n_classes = len(idx2la)
n_vocab = len(idx2w)

In [32]:
# Define model
model = Sequential()
model.add(Embedding(n_vocab,100))
model.add(Convolution1D(64,5,padding='same', activation='relu'))
model.add(Dropout(0.25))
model.add(GRU(100,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 100)         57200     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 64)          32064     
_________________________________________________________________
dropout_4 (Dropout)          (None, None, 64)          0         
_________________________________________________________________
gru_4 (GRU)                  (None, None, 100)         49500     
_________________________________________________________________
time_distributed_4 (TimeDist (None, None, 127)         12827     
Total params: 151,591.0
Trainable params: 151,591.0
Non-trainable params: 0.0
_________________________________________________________________


## Train the model 

In [ ]:
### Training
n_epochs = 100

train_f_scores = []
val_f_scores = []
best_val_f1 = 0

In [ ]:
for i in range(n_epochs):
    print("Epoch {}".format(i))
    
    print("Training =>")
    train_pred_label = []
    avgLoss = 0
    
    bar = progressbar.ProgressBar(max_value=len(train_x))
    for n_batch, sent in bar(enumerate(train_x)):
        label = train_label[n_batch]
        label = np.eye(n_classes)[label][np.newaxis,:]
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #some bug in keras
            loss = model.train_on_batch(sent, label)
            avgLoss += loss

        pred = model.predict_on_batch(sent)
        pred = np.argmax(pred,-1)[0]
        train_pred_label.append(pred)

    avgLoss = avgLoss/n_batch
    
    predword_train = [ list(map(lambda x: idx2la[x], y)) for y in train_pred_label]
    con_dict = conlleval(predword_train, groundtruth_train, words_train, 'r.txt')
    train_f_scores.append(con_dict['f1'])
    print('Loss = {}, Precision = {}, Recall = {}, F1 = {}'.format(avgLoss, con_dict['r'], con_dict['p'], con_dict['f1']))
    
    
    print("Validating =>")
    
    val_pred_label = []
    avgLoss = 0
    
    bar = progressbar.ProgressBar(max_value=len(val_x))
    for n_batch, sent in bar(enumerate(val_x)):
        label = val_label[n_batch]
        label = np.eye(n_classes)[label][np.newaxis,:]
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #some bug in keras
            loss = model.test_on_batch(sent, label)
            avgLoss += loss

        pred = model.predict_on_batch(sent)
        pred = np.argmax(pred,-1)[0]
        val_pred_label.append(pred)

    avgLoss = avgLoss/n_batch
    
    predword_val = [ list(map(lambda x: idx2la[x], y)) for y in val_pred_label]
    con_dict = conlleval(predword_val, groundtruth_val, words_val, 'r.txt')
    val_f_scores.append(con_dict['f1'])
    
    print('Loss = {}, Precision = {}, Recall = {}, F1 = {}'.format(avgLoss, con_dict['r'], con_dict['p'], con_dict['f1']))

    if con_dict['f1'] > best_val_f1:
        best_val_f1 = con_dict['f1']
        open('model_architecture.json','w').write(model.to_json())
        model.save_weights('best_model_weights.h5',overwrite=True)
        print("Best validation F1 score = {}".format(best_val_f1))
    print()
    

N/A% (0 of 19915) |                                         | Elapsed Time: 0:00:00 ETA:  --:--:--

Epoch 0
Training =>


100% (19915 of 19915) |######################################| Elapsed Time: 0:15:28 Time: 0:15:28
N/A% (0 of 4975) |                                          | Elapsed Time: 0:00:00 ETA:  --:--:--

Loss = 0.27927522176196096, Precision = 87.84, Recall = 88.52, F1 = 88.18
Validating =>


100% (4975 of 4975) |########################################| Elapsed Time: 0:01:32 Time: 0:01:32


...

100% (19915 of 19915) |######################################| Elapsed Time: 0:14:32 Time: 0:14:32
  0% (5 of 4975) |                                            | Elapsed Time: 0:00:00 ETA: 0:01:40

Loss = 0.05025689134108131, Precision = 98.87, Recall = 98.96, F1 = 98.91
Validating =>


100% (4975 of 4975) |########################################| Elapsed Time: 0:01:11 Time: 0:01:11
  0% (3 of 19915) |                                           | Elapsed Time: 0:00:00 ETA: 0:16:22

Loss = 0.03824312112403959, Precision = 98.7, Recall = 98.95, F1 = 98.82
Best validation F1 score = 98.82

Epoch 9
Training =>


 30% (6010 of 19915) |############                            | Elapsed Time: 0:06:36 ETA: 0:14:02